# Readme file for Eva's local changes to CTSM folder to get COSMOREA forcing data working

Started 2023-02-28

Instructions copied from @huitang-earth's [scripts_ctsm_region/atm_forcing/cosmo_rea_6km](https://github.com/huitang-earth/scripts_ctsm_region/tree/main/atm_forcing/cosmo_rea_6km) and @ecaas, with additions and modifications by @evalieungh. 

As explained in [Hui's readme file](https://github.com/huitang-earth/scripts_ctsm_region/tree/main/atm_forcing/cosmo_rea_6km), COSMA data are available from <https://opendata.dwd.de/climate_environment/REA/COSMO_REA6/>. Single-site forcing for the VCG sites, subset from the COMSOREA data, is already prepared by Hui on Fram and Saga. Elin downloaded a local copy which is used here. 

Elin's create case command on Saga:
```
/cluster/home/ecaas/CTSM_ecaas_fork/cime/scripts/create_newcase --compset 2000_DATM%COSMOREA_CLM51%BGC_SICE_SOCN_MOSART_SGLC_SWAV --machine saga --run-unsupported --user-mods-dirs /cluster/home/ecaas/ALP1/user_mods --handle-preexisting-dirs r --project nn2806k --case cases/ALP1_I2000.CLM51.COSMOREA.BGC --res CLM_USRDAT
```

To use COSMOREA data, replace the following files with the files provided in "ctsm_config_reg" (works for both regional and any single site simulations):

```
ctsm/components/cdeps/datm/cime_config/namelist_definition_datm.xml
ctsm/components/cdeps/datm/cime_config/stream_definition_datm.xml
ctsm/components/cdeps/datm/cime_config/config_component.xml
```

To use single-site COSMOREA data for VCG, replace the above files with the files provided in "ctsm_config_VCG" (This works only for VCG sites). To better track what has been modified, compare the files with that in "ctsm_config_default".

### Make the changes on NREC LSP repo

Log into NREC virtual machine (see [issue #156](https://github.com/NorESMhub/noresm-land-sites-platform/issues/156) for VM setup).